In [1]:
import utils
import allel
import zarr
import os
import numpy as np
import gzip
import shutil


In [3]:
allel.vcf_to_zarr('../data/org_data/ALL.chr22.phase3_shapeit2_mvncall_integrated_v5b.20130502.genotypes.vcf.gz', '../data/org_data/ALL.chr22.phase3_shapeit2_mvncall_integrated_v5b.20130502.genotypes.zarr', fields='*', overwrite=True)


In [6]:
FILE_NAME_zarr = '../data/org_data/ALL.chr22.phase3_shapeit2_mvncall_integrated_v5b.20130502.genotypes.zarr'
vcf_1KGP = zarr.open_group(FILE_NAME_zarr,mode='r')

In [10]:
with open("samples.txt","w+") as f:
    for sample in vcf_1KGP.samples:
        f.write(f"{sample}\n")

In [7]:
variants = allel.VariantChunkedTable(vcf_1KGP.variants)
variants

<VariantChunkedTable shape=(1103547,) dtype=[('AA', 'O'), ('AC', '<i4', (3,)), ('AF', '<f4', (3,)), ('AFR_AF', '<f4', (3,)), ('ALT', 'O', (3,)), ('AMR_AF', '<f4', (3,)), ('AN', '<i4'), ('CHROM', 'O'), ('CIEND', '<i4', (2,)), ('CIPOS', '<i4', (2,)), ('CS', 'O'), ('DP', '<i4'), ('EAS_AF', '<f4', (3,)), ('END', '<i4'), ('EUR_AF', '<f4', (3,)), ('EX_TARGET', '?'), ('FILTER_PASS', '?'), ('ID', 'O'), ('IMPRECISE', '?'), ('MC', 'O'), ('MEINFO', 'O', (4,)), ('MEND', '<i4'), ('MLEN', '<i4'), ('MSTART', '<i4'), ('MULTI_ALLELIC', '?'), ('NS', '<i4'), ('POS', '<i4'), ('QUAL', '<f4'), ('REF', 'O'), ('SAS_AF', '<f4', (3,)), ('SVLEN', '<i4'), ('SVTYPE', 'O'), ('TSD', 'O'), ('VT', 'O'), ('altlen', '<i4', (3,)), ('is_snp', '?'), ('numalt', '<i4')]
   nbytes=304.2M cbytes=43.9M cratio=6.9
   values=zarr.hierarchy.Group>

In [12]:
def load_lines(filename, header=False):
    header_line = None
    lines = []
    with utils.reading(filename) as fp:
        line_count = 0
        if header:
            header_line = fp.readline().rstrip()
        for line in fp:
            lines.append(line.rstrip())
    return header_line, lines


In [13]:
pannel_legend_file = '../pre_data/hap/chr1_test_true.Omni2.5.legend.gz'
legend_header, legend_lines = load_lines(pannel_legend_file, True)
header_items = legend_header.split()
position_col = utils.get_item_col(header_items, 'position', pannel_legend_file)
positions = [int(line.split()[position_col]) for line in legend_lines]
max_index = len(positions)

In [14]:
unobserved = [] 
observed = []
dictItems = []
af = iter(variants.AF.values)
id = iter(variants.ID.values)
ref_variants = []
with utils.reading(pannel_legend_file) as fp:
    header_items = fp.readline().rstrip().split()
    position_col = utils.get_item_col(header_items, 'position', pannel_legend_file)
    marker_flag_col = utils.get_item_col(header_items, 'array_marker_flag', pannel_legend_file)
    for line in fp:
        items = line.rstrip().split()
        id_tmp = next(id)
        af_tmp = next(af)[0]
        ref_variants.append({'id':id_tmp, 'position': items[1], 'a0': items[2], 'a1':items[3],'a1_freq':af_tmp, 'array_marker_flag':items[4]})

In [15]:
print(len(ref_variants))
print(len(unobserved))
print(len(observed))

305195
300821
4374


In [22]:
def get_index_variant_at_position(position, variants_inp):
    for index, x in enumerate(variants_inp):
        if int(x['position']) == int(position):
            return index, x
    return None

In [42]:
n_split = 2000
start_index, end_index = 0, 0
samples = []
observed_variants_dict = []
for i in range(1,11):
    with gzip.open(f'/mnt/c/Users/vinhdt17/VBDI/Bioinformatics/rnnimp_vn/results/chr1_{i}.legend.gz','r') as legendFile:
        observed_variants = str(legendFile.read()).split("\\n")[1:-1]
        size_obvars = len(observed_variants)
        observed_variants_dict.append({
            'id':id_tmp, 
            'position': items[1],
            'a0': items[2],
            'a1':items[3],
            'a1_freq':af_tmp,
            'array_marker_flag':items[4]
        }) 

    position_end_observed = observed_variants[size_obvars-20].split(' ')[1]
    end_index, _ = get_index_variant_at_position(position_end_observed, ref_variants[start_index:])
    print(start_index, end_index)
    ob_variants = []
    for obvars in observed_variants[size_obvars-20:]:
        _ , variant_tmp = get_index_variant_at_position(obvars.split(' ')[1], ref_variants[start_index:])
        ob_variants.append(variant_tmp)
    samples.append(ref_variants[start_index: end_index] + ob_variants)
    start_index = end_index

0 16385
16385 28215
28215 34314
34314 48153
48153 53424
53424 64894
64894 68105
68105 80020
80020 86604
86604 98110


In [41]:
for x in samples:
    def count_observerd_variant(variants_inp):
        count = 0
        for x in variants_inp:
            if int(x['array_marker_flag']):
                count += 1
        print(count)
    count_observerd_variant(x)

200
85
63
164
68
157
46
173
122
194


In [259]:
samples = [sorted(x, key=lambda k: k['position']) for x in samples]

In [260]:
i = 1
for tmp in samples:
    fileRegion = open(f'/mnt/c/Users/vinhdt17/VBDI/Bioinformatics/rnnimp_vn/region/chr1_{i}.legend','w+')
    fileRegion.write(f"id position a0 a1 a1_freq array_marker_flag\n")
    for sample in tmp:
        if sample['a1_freq'] == 0.0:
            sample['a1_freq'] = 0.00001
        fileRegion.write(f"{sample['id']} {sample['position']} {sample['a0']} {sample['a1']} {sample['a1_freq']} {sample['array_marker_flag']}\n")
    fileRegion.close()
    i += 1

In [261]:
for chr in os.listdir('../region'):
    if 'chr' in chr:
        print(chr)
        index = chr[5:].split(".")[0]
        print(index)
        with open(f'/mnt/c/Users/vinhdt17/VBDI/Bioinformatics/rnnimp_vn/region/{chr}', 'rb') as f_in:
            with gzip.open(f'/mnt/c/Users/vinhdt17/VBDI/Bioinformatics/rnnimp_vn/example_data/model_data/region_{index}.legend.gz', 'wb+') as f_out:
                shutil.copyfileobj(f_in, f_out)


chr1_1.legend
1
chr1_10.legend
10
chr1_2.legend
2
chr1_3.legend
3
chr1_4.legend
4
chr1_5.legend
5
chr1_6.legend
6
chr1_7.legend
7
chr1_8.legend
8
chr1_9.legend
9
